In [1]:
import numpy as np

## Test cases

1. General case (string)
2. No common subsequence
3. One is the subsequence of the other
4. One is empty
5. Both empty
6. Multiple subsequences with the same length

In [2]:
T0 = {
    'input': {
        'seq1': 'serendipitous',
        'seq2': 'precipitation'
    },
    'output': 7
}

T1 = {
    'input': {
        'seq1': [1, 3, 5, 6, 7, 2, 5, 2, 3],
        'seq2': [6, 2, 4, 7, 1, 5, 6, 2, 3]
    },
    'output': 5
}

T2 = {
    'input': {
        'seq1': 'longest',
        'seq2': 'stone'
    },
    'output': 3
}

T3 = {
    'input': {
        'seq1': 'asdfwevad',
        'seq2': 'opkpoiklklj'
    },
    'output': 0
}

T4 = {
    'input': {
        'seq1': 'dense',
        'seq2': 'condensed'
    },
    'output': 5
}

T5 = {
    'input': {
        'seq1': '',
        'seq2': 'opkpoiklklj'
    },
    'output': 0
}

T6 = {
    'input': {
        'seq1': '',
        'seq2': ''
    },
    'output': 0
}

T7 = {
    'input': {
        'seq1': 'abcdef',
        'seq2': 'badcfe'
    },
    'output': 3
}

tests = [T1, T2, T3, T4, T5, T6, T7]

In [13]:
def lcs(seq1, seq2):
    len1, len2 = len(seq1), len(seq2)
    grid = [[0 for i in range(len2+1)] for j in range(len1+1)]

    for i in range(len1):
        for j in range(len2):
            if seq1[i] == seq2[j]:
                grid[i+1][j+1] = 1 + grid[i][j]
            else:
                grid[i+1][j+1] = max(grid[i][j+1], grid[i+1][j])

    return grid[-1][-1]

In [4]:
def lcs_recursive(seq1, seq2, idx1=0, idx2=0):

    if idx1 == len(seq1) or idx2 == len(seq2):
        return 0
    
    elif seq1[idx1] == seq2[idx2]:
        return 1 + lcs_recursive(seq1, seq2, idx1+1, idx2+1)
    
    else:
        option1 = lcs_recursive(seq1, seq2, idx1 +1, idx2)
        option2 = lcs_recursive(seq1, seq2, idx1, idx2 + 1)
        return max(option1, option2)

In [5]:
def lcs_memo(seq1, seq2):
    memo = {}

    def recurse(idx1=0, idx2=0):
        key = (idx1,idx2)

        if key in memo:
            return memo[key]
        elif idx1==len(seq1) or idx2==len(seq2):
            memo[key] = 0
        elif seq1[idx1] == seq2[idx2]:
            memo[key] = 1 + recurse(idx1+1, idx2+1)
        else:
            memo[key] = max(recurse(idx1+1,idx2), recurse(idx1,idx2+1))
        return memo[key]
    return recurse(0,0)

In [14]:
for test in tests:
    result = lcs(**test['input'])
    print(result == test['output'])

True
True
True
True
True
True
True
